In [1]:
import sqlite3
import pandas as pd
import os
import numpy as np

conn = sqlite3.connect("influenza.db")

## Datasets

The data we'll use is publicly available online, and covers the years 2010-2019 in the Chicago area (Cook County, IL). We will import the following CVSs:

In [ ]:
pd.read_csv("weather4.csv").to_sql("weather", conn, index=False, if_exists="replace")
pd.read_csv("google_trends.csv").to_sql("google_trends", conn, index=False, if_exists="replace")
pd.read_csv("vaccination_rates2.csv").to_sql("vaccination_rates", conn, index=False, if_exists="replace")
pd.read_csv("positive_tests.csv").to_sql("positive_tests", conn, index=False, if_exists="replace");

**Weather:** From meteostat.net, specifically the Chicago O'Hare International Airport (ORD) station. It contains over 10 different daily metrics. We'll look at precipitation and temperature only, since those are widely considered the most relevant to flu spread. These are our most granular features, so we will aggregate weekly information to match our target variable intervals.

**Google Trends:** This is a publicly available tool that Google provides. The imported CSV contains search volume by month for different flu-related searches in the Chicago area. We will need to forward-fill data to match our weekly target variable. We need to account for the increase in usage over the 9-year time period, but specific U.S. search volume history is not released by Google. However, it can be found by aggregating monthly volume reports on comscore.com.

**Vaccination Rates:** The CDC releases influenza vaccination data each year. While data isn't available for the Chicago area specifically, we can use Illinois to infer vaccination rates. This data is also monthly, represented by percent of the population currently vaccinated, and so will need to be forward-filled. The vaccination rates given reset in August, so we will have to line it up with our selected season cutoff of 10/1. The dataset itself is large and counts certain groups multiple times. To get the most general rate, we will choose the subset with the highest sample size.

**Positive Tests:** This is our target variable. It was pulled from data.gov and contains weekly positive lab tests. We will shift the data one week backwards to create our target "y" and shift 1, 2, and 3 weeks forwards in order to create lagged case predictors. This database starts later than the others, so we selected 2010 as our starting year. The 2019 end date was chosen to avoid any effects from COVID-19 lockdowns.

Important note: *Most studies estimate that only 5–20% of true infections are represented in testing, so our model will predict positive lab results for the next week (true cases) to avoid any assumptions on real flu levels. Our predictions could be adjusted to estimate actual cases, but we would need to study the relationship between actual flu cases and positive tests in the Chicago area, which we will not do.*

## Data Cleaning/Feature Engineering ##
While SQL is not the most convenient method for this pipeline given the small datasets, it's used to demonstrate technical fluency as well as support scalability.

In [3]:
# In order to find the dimension with the highest sample size, we can query the vaccination_rates table.
pd.read_sql("""
SELECT DISTINCT Dimension
FROM vaccination_rates
WHERE "Season/Survey Year" BETWEEN '2010-11' AND '2019-2020'
ORDER BY "Sample Size"
""", conn);
# The dimension with the highest sample size is >=6 months.

In [4]:
# Create temporary views for cleaned data
conn.execute("""
CREATE TEMP VIEW weather_clean AS
SELECT
    substr(date, 1, 10) AS date_clean,
    tavg,
    prcp
FROM weather
""")

conn.execute("""
CREATE TEMP VIEW google_trends_clean AS
SELECT
    Month || '-01' AS month_date,
    "flu symptoms",
    fever,
    "Cough",
    "sore throat",
    Tamiflu
FROM google_trends
""")

conn.execute("""
CREATE TEMP VIEW vaccination_clean AS
SELECT
    "Season/Survey Year" AS flu_season,
    Month,
    CAST("Estimate (%)" AS REAL) AS vax_rate
FROM vaccination_rates
WHERE Dimension = '>=6 Months'
  AND "Season/Survey Year" BETWEEN '2010-11' AND '2019-20'
""")

conn.execute("""
CREATE TEMP VIEW positive_tests_clean AS
SELECT
    substr(week_start, 7, 4) || '-' || substr(week_start, 1, 2) || '-' || substr(week_start, 4, 2) AS week_start_clean,
    count AS flu_cases
FROM positive_tests
WHERE characteristic_category = 'Influenza Positive'
  AND characteristic_group = 'Total Positive'
""")



In [5]:
# Query to join all cleaned data into a single DataFrame, using Left Joins to weather data, since it is the most granular.
query = """
SELECT
    w.date_clean,
    w.tavg,
    w.prcp,

    g."flu symptoms",
    g.fever,
    g."Cough",
    g."sore throat",
    g.Tamiflu,

    pt.flu_cases,

    vc.vax_rate

FROM weather_clean w

LEFT JOIN google_trends_clean g
    ON w.date_clean = g.month_date

LEFT JOIN positive_tests_clean pt
    ON w.date_clean = pt.week_start_clean

LEFT JOIN vaccination_clean vc
    ON
        strftime('%Y', w.date_clean) || '-' || printf('%02d', CAST(strftime('%m', w.date_clean) AS INT)) = 
        (CASE 
            WHEN vc.Month IN (1, 2, 3) THEN CAST(CAST(substr(vc.flu_season, 1, 4) AS INT) + 1 AS TEXT) || '-' || printf('%02d', vc.Month)
            ELSE substr(vc.flu_season, 1, 4) || '-' || printf('%02d', vc.Month)
         END)
"""
df = pd.read_sql(query, conn)


In [6]:
# Convert date column and sort
df['date_clean'] = pd.to_datetime(df['date_clean'])
df = df.sort_values('date_clean')
df = df.set_index('date_clean')

# Forward-fill vax_rate (monthly)
df['vax_rate'] = df['vax_rate'].ffill()

#Detect drops in vax_rate > 20 and fill values before Oct 1 with last large value (to align reset with seasons)
vax = df['vax_rate']
drops = vax.diff() < -20
drop_indices = df.index[drops]
prev_idx = df.index[0]
for drop_idx in drop_indices:
    # Find last large value before drop
    last_large = vax.loc[:drop_idx].iloc[-2]
    # Find the season year for this drop
    year = drop_idx.year if drop_idx.month >= 10 else drop_idx.year - 1
    oct1 = pd.Timestamp(f'{year}-10-01')
    # Fill values from previous drop (or start) up to Oct 1 with last_large
    mask = (df.index >= prev_idx) & (df.index < oct1)
    df.loc[mask, 'vax_rate'] = last_large
    prev_idx = drop_idx

# Convert vax_rate to fraction
df['vax_rate'] = df['vax_rate'] / 100.0

In [7]:
# Target variable: flu_cases shifted by 7 days in the future.
df['y'] = df['flu_cases'].shift(-7)

In [8]:
# Create lag features for flu_cases
for lag in [7, 14, 21]:
    df[f'flu_cases_lag_{lag}'] = df['flu_cases'].shift(lag)

In [9]:
# Create seasonality features
df['in_season'] = ((df.index.month >= 10) | (df.index.month <= 4)).astype(int)
df['season_year'] = np.where(df.index.month >= 10, df.index.year, df.index.year - 1)
season_start = pd.to_datetime(df['season_year'].astype(str) + '-10-01')

In [10]:
# Work with weekly Google Trends data,
trend_cols = ['Tamiflu', 'fever', 'Cough', 'flu symptoms', 'sore throat']

for col in trend_cols:
    df[col] = df[col].replace('<1', 0.5)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df[trend_cols] = df[trend_cols].ffill()

In [ ]:
# Adjust trend columns for increase in Google usage over seasons (historic U.S. search volume data shows a linear 100% increase from 2010-2019)
min_season = df['season_year'].min()
max_season = df['season_year'].max()
for season in df['season_year'].unique():
    # Linear scaling: 1 for min_season, 0.5 for max_season
    scale = 1 - 0.5 * (season - min_season) / (max_season - min_season)
    df.loc[df['season_year'] == season, trend_cols] = df.loc[df['season_year'] == season, trend_cols] * scale

In [12]:
df[df['tavg'].isna()].index


DatetimeIndex(['2013-01-28'], dtype='datetime64[ns]', name='date_clean', freq=None)

In [13]:
# Fill the few missing temperature values with forward fill, and pcrp with 0
df['tavg'] = df['tavg'].ffill()
df['prcp'] = df['prcp'].fillna(0)

In [14]:
# Calculate weather slopes only for rows where y is not null, before dropping rows with missing y
def weighted_slope_masked(series, mask, window=7):
    import numpy as np
    slopes = np.full(len(series), np.nan)
    weights = np.arange(1, window+1)
    valid_idx = np.where(mask)[0]
    for idx in valid_idx:
        if idx < window:
            continue
        y = series.iloc[idx-window:idx].values
        if np.any(np.isnan(y)):
            continue
        x = np.arange(window)
        w = weights
        x_mean = np.average(x, weights=w)
        y_mean = np.average(y, weights=w)
        numerator = np.sum(w * (x - x_mean) * (y - y_mean))
        denominator = np.sum(w * (x - x_mean)**2)
        slopes[idx] = numerator / denominator if denominator != 0 else 0
    return slopes

mask_y = ~df['y'].isna()
df['tavg_slope'] = weighted_slope_masked(df['tavg'], mask_y)
# For prcp, use sum over previous 7 days (including only if y is not null)
def sum_last_week(series, mask, window=7):
    import numpy as np
    sums = np.full(len(series), np.nan)
    valid_idx = np.where(mask)[0]
    for idx in valid_idx:
        if idx < window:
            continue
        y = series.iloc[idx-window:idx].values
        if np.any(np.isnan(y)):
            continue
        sums[idx] = np.sum(y)
    return sums

df['prcp_sum'] = sum_last_week(df['prcp'], mask_y)

In [15]:
df = df.drop(['flu_cases', 'prcp'], axis=1)
df = df.dropna(subset=['y'])

In [16]:
# Fill NaN values in lagged columns with 0s
lagged_cols = ['flu_cases_lag_7', 'flu_cases_lag_14', 'flu_cases_lag_21']
df[lagged_cols] = df[lagged_cols].fillna(0)

In [17]:
df.head

<bound method NDFrame.head of             tavg  flu symptoms      fever      Cough  sore throat  Tamiflu  \
date_clean                                                                   
2010-09-26  12.3      5.000000  46.000000  27.000000    14.000000     0.00   
2010-10-03   8.8      6.611111  41.555556  26.444444    12.277778     0.00   
2010-10-10  20.6      6.611111  41.555556  26.444444    12.277778     0.00   
2010-10-17  13.1      6.611111  41.555556  26.444444    12.277778     0.00   
2010-10-24  17.5      6.611111  41.555556  26.444444    12.277778     0.00   
...          ...           ...        ...        ...          ...      ...   
2019-08-25  20.3      2.000000  28.000000  19.500000    12.500000     0.25   
2019-09-01  19.4      3.000000  29.000000  25.500000    14.000000     0.50   
2019-09-08  18.4      3.000000  29.000000  25.500000    14.000000     0.50   
2019-09-15  21.4      3.000000  29.000000  25.500000    14.000000     0.50   
2019-09-22  22.3      3.000000  29

In [18]:
df.isna().sum().sum()

0

## Exporting Cleaned Data ##
All datesets were joined correctly, with the matching data index. Features were created, and NaN values were removed. The final cleaned dataset is now ready for modeling and analysis.

In [19]:
df.to_csv("clean_flu_data.csv")
